## Predicción de arriendos - Holanda
  
El dataset se extrajo de [Kaggel](https://www.kaggle.com/datasets/juangesino/netherlands-rent-properties), el dataset *incluye publicaciones sobre las propiedades en arriendo en Holanda*, dada la naturaleza de la tarea, nos encontramos frente a una "Regression Task", en donde la variable a predecir será **rent**.
  
Para esto, utilizaremos la estructura clasica para proyectos de Machine Learning.

*   Exploración de los datos
*   Seleccion de variables
*   Limpieza de datos
*   Pipeline de Machine Learning
*   Evaluación y selección de modelo
*   Ajuste de hiperparámetros
*   Evaluación final & conclusiones

# Getting Started

In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import r2_score,mean_absolute_error

Mounted at /content/drive/


In [4]:
data = pd.read_json('/content/drive/My Drive/Colab Notebooks/ML Portfolio/properties.json', lines=True)


## Exploración de los datos

In [5]:
data.sample(6)

,_id,externalId,areaRaw,areaSqm,city,coverImageUrl,crawlStatus,crawledAt,datesPublished,firstSeenAt,furnish,lastSeenAt,latitude,longitude,postalCode,postedAgo,propertyType,rawAvailability,rent,rentDetail,rentRaw,source,title,url,additionalCosts,additionalCostsRaw,deposit,depositRaw,descriptionNonTranslated,descriptionNonTranslatedRaw,descriptionTranslated,descriptionTranslatedRaw,detailsCrawledAt,energyLabel,gender,internet,isRoomActive,kitchen,living,matchAge,matchAgeBackup,matchCapacity,matchGender,matchGenderBackup,matchLanguages,matchStatus,matchStatusBackup,pageDescription,pageTitle,pets,registrationCost,registrationCostRaw,roommates,shower,smokingInside,toilet,userDisplayName,userId,userLastLoggedOn,userMemberSince,userPhotoUrl,additionalCostsDescription
15750,{'$oid': '5d83faaa536c18a619b16620'},room-1709522,14 m2,14,Den Haag,https://resources.kamernet.nl/image/5706c232-f...,done,{'$date': '2019-09-26T22:10:13.787+0000'},"[{'$date': '2019-09-19T22:01:14.638+0000'}, {'...",{'$date': '2019-09-19T22:01:14.638+0000'},Unfurnished,{'$date': '2019-09-26T22:10:13.916+0000'},52.074465,4.311786,2512BP,1w,Room,01-10-'19 - 01-01-'20,325,Utilities incl.,"€ 325,- Utilities incl.",kamernet,Paviljoensgracht,https://kamernet.nl/en/for-rent/room-den-haag/...,0.0,€ 0,NaN,-,A temporary furnished room is available with W...,A temporary furnished room is available with ...,A temporary furnished room is available with W...,A temporary furnished room is available with ...,{'$date': '2019-09-19T23:31:33.052+0000'},Unknown,Female,Yes,true,Shared,Shared,18 years - 35 years,18 years - 35 years,1 person,Not important,Not important,Not important,Student,Student,"Room for rent in Den Haag, Paviljoensgracht, ...",Room for rent in Den Haag €325 | Kamernet,No,None,-,1,Shared,Yes,Shared,Ron,1445182.0,19-09-2019,29-12-2009,https://resources.kamernet.nl/Content/images/p...,NaN
24352,{'$oid': '5dcc8b3c8e7e9f1f4fcd02d7'},room-1723685,15 m2,15,Leiden,https://resources.kamernet.nl/image/69e58bbf-d...,done,{'$date': '2019-11-18T03:11:09.315+0000'},"[{'$date': '2019-11-13T23:01:16.127+0000'}, {'...",{'$date': '2019-11-13T23:01:16.127+0000'},Uncarpeted,{'$date': '2019-11-18T03:11:09.437+0000'},52.161023,4.478619,2312AE,4d,Room,13-12-'19 - Indefinite period,330,Utilities incl.,"€ 330,- Utilities incl.",kamernet,Morsweg,https://kamernet.nl/en/for-rent/room-leiden/mo...,0.0,€ 0,NaN,-,Geen reistijdvoorrang nodig! Ben jij een leuke...,Geen reistijdvoorrang nodig! <br><br>Ben jij ...,,,{'$date': '2019-11-13T23:32:41.318+0000'},Unknown,Male,No,true,Shared,Shared,18 years - 24 years,18 years - 24 years,1 person,Male,Male,Not important,Student,Student,"Room for rent in Leiden, Morsweg, for €330 a ...",Room for rent in Leiden €330 | Kamernet,No,None,NA,7,Shared,No,Shared,Rozemarijn,4124642.0,13-11-2019,20-04-2017,https://resources.kamernet.nl/image/9b5476e7-9...,NaN
33034,{'$oid': '5e0932d9ee0c17b578567daa'},room-1740969,19 m2,19,Den Haag,https://resources.kamernet.nl/image/7cd52e08-3...,done,{'$date': '2019-12-31T23:13:52.917+0000'},"[{'$date': '2019-12-29T23:12:25.439+0000'}, {'...",{'$date': '2019-12-29T23:12:25.439+0000'},Furnished,{'$date': '2019-12-31T23:13:53.063+0000'},52.055732,4.297629,2531CB,21 dec '19,Room,04-01-'20 - Indefinite period,550,Utilities incl.,"€ 550,- Utilities incl.",kamernet,Vier Heemskinderenstraat,https://kamernet.nl/en/for-rent/room-den-haag/...,NaN,NA,NaN,-,De kamer word gemeubileerd verhuurd ook Is er ...,De kamer word gemeubileerd verhuurd ook <br>I...,,,{'$date': '2019-12-30T07:32:30.779+0000'},Unknown,Unknown,Yes,true,Shared,None,18 years - 35 years,18 years - 35 years,3 persons,Not important,Not important,Not important,"Student, Working student, Working","Student, Working student, Working","Room for rent in Den Haag, Vier Heemskinderen...",Room for rent in Den Haag €550 | Kamernet,No,None,NA,3,Shared,No,Shared,Frans,4799861.0,29-12-2019,21-12-2019,https://resources.kamernet.nl/Content/images/p...,NA
38685,{'$oid': '5e2e1a5acf5f457d815461

In [6]:
data.describe()

,areaSqm,latitude,longitude,rent,additionalCosts,deposit,userId
count,46722.000000,46722.000000,46722.000000,46722.000000,14301.000000,27704.000000,4.662200e+04
mean,31.616626,52.201846,5.314911,667.745516,69.648346,713.447083,3.425398e+06
std,29.863150,0.517203,0.798989,416.667339,132.432817,942.256946,1.327936e+06
min,6.000000,50.770041,3.410016,1.000000,0.000000,0.000000,6.240000e+02
25%,14.000000,51.925491,4.711688,395.000000,0.000000,360.000000,2.894253e+06
50%,20.000000,52.162498,5.082988,550.000000,13.000000,500.000000,3.934324e+06
75%,40.000000,52.370420,5.896362,800.000000,99.000000,850.000000,4.515906e+06
max,675.000000,53.434608,7.206637,5999.000000,5000.000000,107514.000000,4.854533e+06


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46722 entries, 0 to 46721
Data columns (total 62 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   _id                          46722 non-null  object 
 1   externalId                   46722 non-null  object 
 2   areaRaw                      46722 non-null  object 
 3   areaSqm                      46722 non-null  int64  
 4   city                         46722 non-null  object 
 5   coverImageUrl                46722 non-null  object 
 6   crawlStatus                  46722 non-null  object 
 7   crawledAt                    46722 non-null  object 
 8   datesPublished               46722 non-null  object 
 9   firstSeenAt                  46722 non-null  object 
 10  furnish                      46722 non-null  object 
 11  lastSeenAt                   46722 non-null  object 
 12  latitude                     46722 non-null  float64
 13  longitude       

El dataset cuenta con 60+ variables, de las cuales solo 7 son cuantitativas: "rent", "areaSqm", "additional cost", "deposit" incluyendo  "latitud" & "longitud", "userId". Las demás son cualitativas y debemos analizar **la relevancia de la variable** y la **cantidad de observaciones**, este análisis contempla:

*   Features o variables duplicadas: Al revisar los valores de las variables, vemos que algunas se encuentran en formato raw y otras procesado, como es el caso de ***areaRaw*** y ***areaSqm***.

*   Features o variables irrelevantes: El caso de las variables "userLastLoggedOn" o "userId", corresponden a comportamientos en el sistema, por lo que carecen de importancia para el modelo.


Luego, al analizar la calidad de la data, vemos que "userId", "descriptionNonTranslated", "depositRaw", entre otras, tienen 100 nulls exactos, por lo que puede existir un patron:

```
index_isna=[index for index, is_na in data['userId'].isna().items() if is_na]
data.loc[index_isna]
```

Al revisar, la variable **crawlStatus** aparece como "unavailable", es decir, que ha habido un problema con el proceso  de recolección de la data por lo que  eliminaré dichas observaciones.



## Selección de variables



In [8]:
data.columns

Index(['_id', 'externalId', 'areaRaw', 'areaSqm', 'city', 'coverImageUrl',
       'crawlStatus', 'crawledAt', 'datesPublished', 'firstSeenAt', 'furnish',
       'lastSeenAt', 'latitude', 'longitude', 'postalCode', 'postedAgo',
       'propertyType', 'rawAvailability', 'rent', 'rentDetail', 'rentRaw',
       'source', 'title', 'url', 'additionalCosts', 'additionalCostsRaw',
       'deposit', 'depositRaw', 'descriptionNonTranslated',
       'descriptionNonTranslatedRaw', 'descriptionTranslated',
       'descriptionTranslatedRaw', 'detailsCrawledAt', 'energyLabel', 'gender',
       'internet', 'isRoomActive', 'kitchen', 'living', 'matchAge',
       'matchAgeBackup', 'matchCapacity', 'matchGender', 'matchGenderBackup',
       'matchLanguages', 'matchStatus', 'matchStatusBackup', 'pageDescription',
       'pageTitle', 'pets', 'registrationCost', 'registrationCostRaw',
       'roommates', 'shower', 'smokingInside', 'toilet', 'userDisplayName',
       'userId', 'userLastLoggedOn', 'userMember

In [9]:
def preprocess_feature(df):
    df = df.copy()

    bad_rows = df[df['crawlStatus'] == 'unavailable'].index
    df = df.drop(bad_rows, axis=0).reset_index(drop=True)
    df = df[[
        'areaSqm',
        'city',
        'furnish',
        'latitude',
        'longitude',
        'propertyType',
        'rent',
        'internet',
        'kitchen',
        'living',
        'shower',
        'smokingInside',
        'toilet'
    ]]
    return df


## Limpieza de los **datos**

Al ver los valores que toman las variables categoricas, nos damos cuenta que existen errores en la definición de los missings.

```
{column:new_data[column].unique() for column in new_data.select_dtypes(include=['object']).columns[1:]}
```

Además, vemos que una cantidad de observaciones no tienen valor.   


```
{col: val * 100 for col, val in new_data.isnull().mean().to_dict().items() if val !=0 }
```
Existen muchas técnicas para mejorar la calidad de la data, desde ML hasta estadisticas, en este caso trabajaremos con la estadistica y dado que son variables categoricas debemos utilizar la moda.

In [10]:
def preprocess_data(df):
    df=preprocess_feature(df)
    #Missings mal codificados
    df = df.replace({'': np.NaN, 'Unknown': np.NaN})
    #Remplazo de datos faltantes
    missing_value_columns = df.columns[df.isna().sum() > 0]
    for column in missing_value_columns:
        df[column] = df[column].fillna(df[column].mode()[0])
    X=df.drop('rent',axis=1)
    Y=df['rent']
    return X,Y

In [11]:
X,Y = preprocess_data(data)

In [12]:
{column:len(X[column].unique()) for column in X.select_dtypes(include=['object']).columns}

{'city': 737,
 'furnish': 3,
 'propertyType': 5,
 'internet': 2,
 'kitchen': 3,
 'living': 3,
 'shower': 3,
 'smokingInside': 3,
 'toilet': 3}

In [13]:
{column:X[column].unique() for column in X.select_dtypes(include=['object']).columns[1:]}

{'furnish': array(['Unfurnished', 'Furnished', 'Uncarpeted'], dtype=object),
 'propertyType': array(['Room', 'Studio', 'Apartment', 'Anti-squat', 'Student residence'],
       dtype=object),
 'internet': array(['Yes', 'No'], dtype=object),
 'kitchen': array(['Shared', 'Own', 'None'], dtype=object),
 'living': array(['None', 'Own', 'Shared'], dtype=object),
 'shower': array(['Shared', 'Own', 'None'], dtype=object),
 'smokingInside': array(['No', 'Yes', 'Not important'], dtype=object),
 'toilet': array(['Shared', 'Own', 'None'], dtype=object)}

## Pipeline de Machine Learning

Las variables *categóricas* son **nominales**, es decir, carecen de un orden. Para incluirlas en el modelo, debemos implementar una estrategia de **codificacion** dependiendo si son binarias o no.

In [14]:
# @title Preprocessor - Codificación de las variables
binary_features = [
    'internet'
]
nominal_features = [
    'city',
    'furnish',
    'propertyType',
    'kitchen',
    'living',
    'shower',
    'smokingInside',
    'toilet'
]
# Transformers según tipo de variable
binary_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(categories=[['No','Yes']])) #0 es No y 1 es Yes
])
nominal_transformer = Pipeline(steps=[
    ('nominal', OneHotEncoder(handle_unknown='ignore')) # handle_unknown permite procesar valores que no estaban en el test, IE: City
])

preprocessor = ColumnTransformer(transformers=[
    ('binary', binary_transformer, binary_features),
    ('nominal', nominal_transformer, nominal_features)
], remainder='passthrough') #las variables que no se procesan, igual se mantienen


In [15]:
# @title Set de entrenamiento y prueba
def train_test(X,Y):
  X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=.8,random_state=20,shuffle=True)
  return X_train,X_test,Y_train,Y_test


Aunque el utilizar K-Fold Cross-Validation proporciona una evaluación más robusta y confiable porque cada observación se utiliza tanto para training como para testing, lo que **reduce la variabilidad en las estimaciones de rendimiento**, utilizaremos el split "Train & Test", para la primera evaluación de los modelos.

In [16]:
X_train,X_test,Y_train,Y_test=train_test(X[0:len(X)-1],Y[0:len(Y)-1])

## Selección y evaluación de modelos

Para predecir el valor de la renta, utilizaremos tres modelos de regresión. Evaluaremos cada uno de ellos y seleccionaremos el que tenga el mejor desempeño para avanzar a una etapa de fine-tuning, donde optimizaremos los parámetros del modelo para maximizar su rendimiento. Los modelos seleccionados son:

*   Decision Tree
*   Random Forest
*   XGBoost

La razón por la que utilizaremos estos modelos es por su capacidad para manejar relaciones complejas y no lineales en los datos, así como su adaptabilidad a  datasets que se han intervenido o valores faltantes.

In [17]:
class ModelEvaluator(BaseEstimator, TransformerMixin):
    def __init__(self, regressor):
        self.regressor = regressor

    def fit(self, X, y=None):
        self.regressor.fit(X, y)
        return self

    def transform(self, X):
        return self

    def predict(self, X):
        return self.regressor.predict(X)


    def score(self, X, y):
        y_pred = self.regressor.predict(X)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)
        print(f"Mean Absolut Error: {mae}")
        print("R2: {:.2f}%".format(r2 * 100))
        return r2

model_RF = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', ModelEvaluator(RandomForestRegressor()))
])
model_XG = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', ModelEvaluator(GradientBoostingRegressor()))
])
model_DT = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', ModelEvaluator(DecisionTreeRegressor()))
])


In [18]:
# @title Selección de modelo: Random Forest
# Random Forest
model_RF.fit(X_train, Y_train)
print("Random Forest Regressor:")
model_RF.score(X_test, Y_test)
# Gradient Boosting
model_XG.fit(X_train, Y_train)
print("Gradient Boosting Regressor:")
model_XG.score(X_test, Y_test)
# Decision Tree
model_DT.fit(X_train, Y_train)
print("Decision Tree Regressor:")
model_DT.score(X_test, Y_test)

Random Forest Regressor:
Mean Absolut Error: 86.0561889444144
R2: 83.43%
Gradient Boosting Regressor:
Mean Absolut Error: 121.07808344815221
R2: 77.26%
Decision Tree Regressor:
Mean Absolut Error: 101.38559122566912
R2: 74.79%


0.7478611236536796

A partir de estos resultados, utilizaremos el modelo de Random Forest para la etapa de fine-tune. Esta elección se debe al menor error absoluto medio (MAE) y el mayor coeficiente de determinación (R2).

## Ajuste de hiperparámetros
El proceso de fine-tuning consta de tres etapas:

*   Separación de los datos en conjuntos de entrenamiento y prueba.
*   Entrenamiento y evaluación del modelo para hiperparámetro previamente definido mediante validación cruzada K-Fold, en nuestro caso, la metrica de evaluación será R2.
*   Evaluación final del modelo con el hiperparametro de mejor rendimiento, esta evaluación se realiza a partir del conjunto de prueba, el cual no se ha utilizadoen el entrenamiento, para medir la asertividad real del modelo.

Para el caso de Random Forest, el hiperparámetro que ajustaremos es el número de árboles [50, 100, 140], en donde cada modelo se evaluará con un k=4.

In [19]:
param_grid = {'regressor__regressor__n_estimators': [50, 100, 140]}
grid_search = GridSearchCV(model_RF, param_grid, cv=4)

# Ajustar el modelo usando GridSearchCV
grid_search.fit(X_train, Y_train)

Mean Absolut Error: 90.48483929053562
R2: 80.65%
Mean Absolut Error: 87.93772990628081
R2: 84.12%
Mean Absolut Error: 87.50380242163853
R2: 86.17%
Mean Absolut Error: 87.98429381367218
R2: 84.91%
Mean Absolut Error: 89.74589459871481
R2: 80.93%
Mean Absolut Error: 87.61193678839959
R2: 84.05%
Mean Absolut Error: 86.92215299681904
R2: 86.47%
Mean Absolut Error: 87.85883489906465
R2: 84.79%
Mean Absolut Error: 89.72993913180578
R2: 80.93%
Mean Absolut Error: 87.1132547552556
R2: 84.34%
Mean Absolut Error: 86.94484937949186
R2: 86.30%
Mean Absolut Error: 87.79026370343735
R2: 84.80%


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('binary',
                                                                         Pipeline(steps=[('ordinal',
                                                                                          OrdinalEncoder(categories=[['No',
                                                                                                                      'Yes']]))]),
                                                                         ['internet']),
                                                                        ('nominal',
                                                                         Pipeline(steps=[('nominal',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['city',
                                                                          'furnish',
                                                                          'propertyType',
                                                                          'kitchen',
                                                                          'living',
                                                                          'shower',
                                                                          'smokingInside',
                                                                          'toilet'])])),
                                       ('regressor',
                                        ModelEvaluator(regressor=RandomForestRegressor()))]),
             param_grid={'regressor__regressor__n_estimators': [50, 100, 140]})

In [20]:
# Imprimir los mejores parámetros
print("Cantidad optima de arboles:", grid_search.best_params_.get('regressor__regressor__n_estimators'))

# Evaluar el mejor modelo
best_model = grid_search.best_estimator_
print("Random Forest Regressor:")
best_model.score(X_train,Y_train)

Cantidad optima de arboles: 140
Random Forest Regressor:
Mean Absolut Error: 33.25252132944889
R2: 97.77%


0.9776842255820378

Tras el proceso de fine-tune del hiperparámetro "cantidad de árboles" utilizada por el algoritmo, vemos que la medida R2 o **capacidad del modelo de predecir de valor de la renta a partir de las variables utilizadas**, en el conjunto de datos de entrenamiento viene explicada en un 97.77%.

Para obtener la real capacidad predictiva del modelo, se debe evaluar a partir de un conjunto de datos que no haya sido utilizado por el modelo en la fase de entrenamiento, como es el set de prueba.

## Evaluación final & conclusiones

Finalmente, vemos que la real capacidad predictiva del modelo optimizado es de un 83.45%



In [21]:
best_model.score(X_test, Y_test)

Mean Absolut Error: 85.75296169851423
R2: 83.45%


0.8345300075182729

Si bien los modelos de ensamblaje funcionan sin necesidad de normalizar las variables, una mejora al proceso realizado anteriormente sería integrar un escalador (scaler) al pipeline. Esto podría mejorar la precisión, especialmente del modelo XGBoost, que puede beneficiarse significativamente de la normalización de las características.